In [1]:
#import modules
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import sys
import h5py
import tensorflow.compat.v1 as tf
import tensorflow as tf2

from DlModels import DlModels
from TrainDL import TrainDL
from DataLoader import DataLoader

In [2]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


False

In [ ]:
tf.config.list_physical_devices('GPU')
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)
tf.keras.backend.set_floatx('float32') #for memory use

In [5]:
#test batching
data_loader = DataLoader('event_frame_data.hdf5')

event_images, depth_images, mask_1, mask_2 = data_loader.load_all()

fig = plt.figure()
ax = fig.add_subplot(2, 2, 1)
ax.imshow(np.array(event_images[10][:,:,0:3]))
ax.autoscale(True)
ax2 = fig.add_subplot(2, 2, 2, sharex=ax, sharey=ax)
ax2.imshow(np.array(depth_images[10]))
ax2.autoscale(True)
ax3 = fig.add_subplot(2, 2, 3, sharex=ax, sharey=ax)
ax3.imshow(np.array(mask_1[10]))
ax3.autoscale(True)
ax4 = fig.add_subplot(2, 2, 4, sharex=ax, sharey=ax)
ax4.imshow(np.array(mask_2[10]))
ax4.autoscale(True)
plt.show()


/home/abdulla/Insync/KU/codes/AI/DL_for_DVS/contact_est/DataLoader.py:9: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  self.data_ = h5py.File(filename)


AttributeError: 'DataLoader' object has no attribute 'load_all'

In [2]:
epochs = 10
batch_size = 10
base_learning_rate = 0.01
learning_decay_rate = 30
learning_decay_factor = 0.5
data_loader = DataLoader('processed_data_10_Hz.hdf5')

tf.compat.v1.disable_eager_execution()
input_image = tf.placeholder(tf.float32, [None, None, 260, 346, 3], name='input_image')
grount_truth = tf.placeholder(tf.float32, [None, 6])
learning_rate = tf.placeholder(tf.float32, shape=(), name='learning_rate')

sess = tf.Session()

# Build NN using load_vgg, layers, and optimize function
nn_last_layer = DlModels.Conv2dx2LSTMFC(input_image, encoder=DlModels.Conv2Dx1, N_outputs=6)

train_op, loss = TrainDL.SoftEntropy(nn_last_layer, grount_truth, learning_rate)

#accuracy
correct_prediction = tf.equal(tf.argmax(nn_last_layer, 1), tf.argmax(grount_truth, 1))
accuracy_op = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='accuracy_operation')

# Train NN using the train_nn function
TrainDL.train_nn(sess, epochs, batch_size, data_loader, accuracy_op, train_op, loss, input_image, grount_truth, learning_rate, base_learning_rate, learning_decay_rate, learning_decay_factor)
        
# save trained model
saver = tf.train.Saver()
saver.save(sess, './model')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
/home/abdulla/Insync/KU/codes/AI/DL_for_DVS/contact_est/DataLoader.py:10: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  self.data_ = h5py.File(filename)
Training...

EPOCH 1 ...
Batch 1 ...
Accuracy [0.5] ...
Batch 1 ...
Accuracy [0.1] ...
Batch 1 ...
Accuracy [0.2] ...
Batch 1 ...
Accuracy [0.2] ...
Batch 1 ...
Accuracy [0.2] ...
Batch 1 ...
Accuracy [0.0] ...
Batch 1 ...
Accuracy [0.2] ...
Batch 1 ...
Accuracy [0.2] ...
Batch 1 ...
Accuracy [0.3] ...
Batch 1 ...
Accuracy [0.0] ...
Batch 1 ...
Accuracy [0.1] ...
Batch 1 ...
